<a href="https://colab.research.google.com/github/ashivashankars/archana_Autogluon_repo/blob/main/AutoGluon_for_Kaggle(California_House_Prices)_competitions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##California House Prices
Predict California sales prices

Description
The task is to predict house sale prices based on the house information, such as # of bedrooms, living areas, locations, near-by schools, and the seller summary. The data consist of houses sold in California on 2020, with houses in the test dataset sold after the ones in the training dataset. Also the private leaderboard houses were sold after the ones in the public leaderboard.



##1. Run Bash command: pip install kaggle!

In [2]:
!pip -q install -U kaggle

##2. Navigate to: https://www.kaggle.com/account and create an account (if necessary). Then , click on “Create New API Token” and move downloaded file to this location on your machine: ~/.kaggle/kaggle.json. For troubleshooting, see Kaggle API instructions.

In [3]:
from google.colab import files
files.upload()  # choose kaggle.json you downloaded from Kaggle > Account > Create New API Token

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


##3. To download data programmatically: Execute this Bash command in your terminal:

kaggle competitions download -c [COMPETITION]

Here, [COMPETITION] should be replaced by the name of the competition you wish to enter. Alternatively, you can download data manually: Just navigate to website of the Kaggle competition you wish to enter, click “Download All”, and accept the competition’s terms.

In [4]:
import json, os
p = os.path.expanduser('~/.kaggle/kaggle.json')
creds = json.load(open(p))
print("Using Kaggle account:", creds["username"])


Using Kaggle account: archanshivashankar


In [5]:
!kaggle competitions files -c california-house-prices


name                         size  creationDate                
---------------------  ----------  --------------------------  
sample_submission.csv      253022  2021-03-19 22:16:47.263000  
test.csv                 36182130  2021-03-19 22:16:47.263000  
train.csv                53361674  2021-03-19 22:16:47.263000  


In [6]:
!kaggle competitions download -c california-house-prices -p /content -w
# (The competition slug is all lowercase: ieee-fraud-detection)


  0% 0.00/29.5M [00:00<?, ?B/s]
100% 29.5M/29.5M [00:00<00:00, 1.61GB/s]


##4. If the competition’s training data is comprised of multiple CSV files, use pandas to properly merge/join them into a single data table where rows = training examples, columns = features.

In [7]:
import zipfile
import os

zip_file_path = '/content/california-house-prices.zip'
destination_path = '/content/'

if os.path.exists(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(destination_path)
    print(f"Extracted {zip_file_path} to {destination_path}")
else:
    print(f"Error: {zip_file_path} not found. Please ensure the competition data is downloaded.")

Extracted /content/california-house-prices.zip to /content/


4(a): we first load the competition’s training data into Python:

In [9]:
!pip install -q autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 26.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.9/454.9 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 17.0 MB/s eta 0:0

In [10]:
import pandas as pd
import numpy as np
from pathlib import Path
from autogluon.tabular import TabularPredictor

directory = Path("/content")  # directory where you have downloaded the data CSV files from the competition
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
save_path = directory/'AutoGluonModels/'  # where to store trained models

test = pd.read_csv(directory/'test.csv')
train = pd.read_csv(directory/'train.csv')

4(b):Since the training data for this competition is comprised of multiple CSV files, we just first join them into a single large table (with rows = examples, columns = features) before applying AutoGluon:

In [11]:
train_data = pd.merge(train, test, on='Id', how='left')

4(c):we specify the presets argument to maximize AutoGluon’s predictive accuracy which usually requires that you run fit()

In [12]:
predictor = TabularPredictor(label='Sold Price', path=save_path, verbosity=3).fit(
    train_data, presets='best_quality', time_limit=3600
)

results = predictor.fit_summary()

Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          12
GPU Count:          1
Memory Avail:       81.17 GB / 83.47 GB (97.2%)
Disk Space Avail:   189.97 GB / 235.68 GB (80.6%)
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True, 'num_bag_sets': 1}
Full kwargs:
{'_experimental_dynamic_hyperparameters': False,
 '_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'delay_bag_sets': False,
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_callbacks': False,
             'enable_ray_logging': True,
             'holdout_data': None,
             'ho

(_ray_fit pid=5444) [50]	valid_set's rmse: 809843
(_ray_fit pid=5444) [100]	valid_set's rmse: 739735
(_ray_fit pid=5442) [50]	valid_set's rmse: 639413
(_ray_fit pid=5440) [50]	valid_set's rmse: 680041
(_ray_fit pid=5443) [50]	valid_set's rmse: 676436 [repeated 8x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=5443) [100]	valid_set's rmse: 578437 [repeated 10x across cluster]
(_ray_fit pid=5437) [300]	valid_set's rmse: 711839 [repeated 11x across cluster]
(_ray_fit pid=5437) [350]	valid_set's rmse: 710404 [repeated 12x across cluster]
(_ray_fit pid=5438) [450]	valid_set's rmse: 942678 [repeated 11x across cluster]
(_ray_fit pid=5439) [200]	valid_set's rmse: 760880 [repeated 12x across cluster]


(_ray_fit pid=5440) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L1/S1F5/model.pkl
(_ray_fit pid=5444) 	Fitting 10000 rounds... Hyperparameters: {'learning_rate': 0.05, 'extra_trees': True} [repeated 7x across cluster]


(_ray_fit pid=5438) [650]	valid_set's rmse: 936757 [repeated 10x across cluster]
(_ray_fit pid=5438) [750]	valid_set's rmse: 936911 [repeated 12x across cluster]


(_ray_fit pid=5439) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L1/S1F4/model.pkl
(_ray_fit pid=5443) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L1/S1F6/model.pkl


...

(_ray_fit pid=8498) [1600]	valid_set's rmse: 800961 [repeated 5x across cluster]
(_ray_fit pid=8498) [1700]	valid_set's rmse: 800828 [repeated 4x across cluster]
(_ray_fit pid=8495) [1350]	valid_set's rmse: 692627 [repeated 5x across cluster]
(_ray_fit pid=8495) [1450]	valid_set's rmse: 692671 [repeated 4x across cluster]
(_ray_fit pid=8495) [1550]	valid_set's rmse: 691977 [repeated 4x across cluster]
(_ray_fit pid=8495) [1650]	valid_set's rmse: 691971 [repeated 4x across cluster]
(_ray_fit pid=8495) [1750]	valid_set's rmse: 691790 [repeated 4x across cluster]
(_ray_fit pid=8495) [1850]	valid_set's rmse: 691457 [repeated 4x across cluster]
(_ray_fit pid=8495) [1950]	valid_set's rmse: 691179 [repeated 5x across cluster]
(_ray_fit pid=8495) [2050]	valid_set's rmse: 691038 [repeated 4x across cluster]


(_ray_fit pid=8498) 	Ran out of time, early stopping on iteration 2671. Best iteration is:
(_ray_fit pid=8498) 	[2670]	valid_set's rmse: 799692


(_ray_fit pid=8495) [2200]	valid_set's rmse: 690829 [repeated 5x across cluster]


(_ray_fit pid=8498) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L2/S1F3/model.pkl
(_ray_fit pid=8495) 	Ran out of time, early stopping on iteration 2212. Best iteration is:
(_ray_fit pid=8495) 	[2210]	valid_set's rmse: 690720
(_dystack pid=3989) 	-819118.2313	 = Validation score   (-root_mean_squared_error)
(_dystack pid=3989) 	190.77s	 = Training   runtime
(_dystack pid=3989) 	9.77s	 = Validation runtime
(_dystack pid=3989) 	199.2	 = Inference  throughput (rows/s | 5271 batch size)
(_dystack pid=3989) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/trainer.pkl
(_dystack pid=3989) Fitting model: LightGBM_BAG_L2 ... Training model for up to 34.64s of the 34.35s of remaining time.
(_dystack pid=3989) 	Fitting LightGBM_BAG_L2 with 'num_gpus': 0, 'num_cpus': 12
(_dystack pid=3989) Loading: /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBM_BAG_L2/utils/model_template.pkl
(_dystack pid=3989) Saving /content/AutoGluonModels/ds_sub_fit/

*** Summary of fit() ***
Estimated performance of each model:
                    model      score_val              eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0     WeightedEnsemble_L3 -679505.719289  root_mean_squared_error     238.919284  5907.253689                0.001033           0.027802            3       True          5
1     WeightedEnsemble_L2 -679505.719289  root_mean_squared_error     238.919351  5907.256849                0.001099           0.030962            2       True          4
2         LightGBM_BAG_L1 -699599.455560  root_mean_squared_error       6.983701   228.079005                6.983701         228.079005            1       True          2
3  RandomForestMSE_BAG_L1 -715159.610745  root_mean_squared_error     207.460877  5132.103482              207.460877        5132.103482            1       True          3
4       LightGBMXT_BAG_L1 -788328.889662  root_mean_squared_error      24.4736

In [13]:
import pandas as pd
import numpy as np
from pathlib import Path

directory = Path("/content")
ID_COL = "Id" # Use 'Id' as the ID column for this competition
LABEL  = "Sold Price"  # your training label for regression

# --- load test ---
# Since the test data is already loaded in a previous cell (9vAr2fH21fHp) as 'test',
# we can directly use it.
test_data = test.copy()

# drop accidental index columns - This line might not be necessary depending on how test.csv is read,
# but it's kept for robustness if such columns appear.
test_data = test_data.loc[:, ~test_data.columns.astype(str).str.startswith("Unnamed")]

# don't pass target - In regression, the target column is not in the test data.
# We don't need to explicitly drop it here as it's not expected to be in the test data.

# --- align to exactly the features used in training ---
# Get the list of features used by the trained predictor
features = predictor.features()

# Ensure the test data has the same columns as the training data used by the predictor
# and in the same order. New columns will be NaN if not in test_data.
test_data = test_data.reindex(columns=features)

# --- make dtypes NumPy-friendly; replace pd.NA with np.nan ---
test_data = test_data.replace({pd.NA: np.nan})

for col in test_data.columns:
    dt = test_data[col].dtype
    # pandas nullable integers (Int64/Int32/Int16) -> float (allows np.nan)
    if str(dt) in {"Int64", "Int32", "Int16", "UInt64", "UInt32", "UInt16"}:
        test_data[col] = test_data[col].astype("float32")
    # pandas nullable boolean -> float (0.0/1.0 + nan)
    elif str(dt) == "boolean":
        test_data[col] = test_data[col].astype("float32")
    # pandas string dtype -> plain object (np.nan-compatible)
    elif pd.api.types.is_string_dtype(dt):
        test_data[col] = test_data[col].astype("object")

# --- predict ---
# For regression, we use predict instead of predict_proba
y_pred = predictor.predict(test_data)

Loading: /content/AutoGluonModels/models/LightGBMXT_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/LightGBM_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/RandomForestMSE_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/WeightedEnsemble_L2/model.pkl


In [15]:
y_pred.head(5)

,Sold Price
0,474998.84375
1,474998.84375
2,474998.84375
3,474998.84375
4,474998.84375


For binary classification tasks, you can see which class AutoGluon’s predicted probabilities correspond to via:

In [16]:
predictor.positive_class

For multiclass classification tasks, you can see which classes AutoGluon’s predicted probabilities correspond to via:

In [17]:
predictor.class_labels  # classes in this list correspond to columns of predict_proba() output

Now, let’s get prediction probabilities for the entire test data, while only getting the positive class predictions by specifying:

In [22]:
y_pred = predictor.predict(test_data)

Loading: /content/AutoGluonModels/models/LightGBMXT_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/LightGBM_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/RandomForestMSE_BAG_L1/model.pkl
Loading: /content/AutoGluonModels/models/WeightedEnsemble_L2/model.pkl


Now that we have made a prediction for each row in the test dataset, we can submit these predictions to Kaggle. Most Kaggle competitions provide a sample submission file, in which you can simply overwrite the sample predictions with your own as we do below:

In [25]:
submission = pd.read_csv(directory/'sample_submission.csv')
submission['Sold Price'] = y_pred
submission.head()
submission.to_csv(directory/'archie_submission.csv', index=False)

##To submit your predictions to Kaggle, you can run the following command in your terminal

In [26]:
!kaggle competitions submit -c california-house-prices -f archie_submission.csv -m "my first submission"

100% 494k/494k [00:02<00:00, 201kB/s]
Successfully submitted to California House Prices